# Preapre

In [114]:
import wrds
import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import preprocessing, tree
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import permutation_importance
pd.set_option('display.max_columns', None)
import statsmodels.api as sm
from statsmodels.api import add_constant
from sklearn.metrics import r2_score
from CompanyTextSentimentAnalyzer import TextSentimentAnalyser


import warnings
warnings.filterwarnings("ignore")

In [115]:
db = wrds.Connection(wrds_username = 'rotmanren') # Change to user's username

Loading library list...
Done


# Data injection

In [118]:
# Load all data
startYear = 2009
# Accruals need data from 2009 to obtain changes
endYear = 2022
dataquery1 = f"""
            SELECT a.DATADATE, a.cik, a.GVKEY, a.tic, a.fyear, a.ib, a.spi, a.at, a.dvc, a.act, a.che, a.lct, a.dlc, a.ni, a.txp, a.revt, a.dp, a.csho, a.ceq, a.ivao, a.lt, a.ajex, a.dltt, a.ivst, a.pstk, b.cshom, b.prccm
            FROM comp.funda as a
            JOIN comp_na_daily_all.secm as b
            ON a.tic = b.tic AND a.fyear = b.cyear
            WHERE consol = 'C'
            AND popsrc = 'D'
            AND indfmt = 'INDL'
            AND datafmt = 'STD' 
            AND fyear BETWEEN '{startYear}' AND '{endYear}'
            AND b.cmth = 6.0
        """
inforaw = db.raw_sql(dataquery1)
# Fill NA for selected variables 
cols = ['spi', 'dvc', 'che', 'lct', 'dlc', 'txp', 'dp', 'ceq', 'ivao', 'lt', 'dltt', 'ivst', 'pstk']
inforaw[cols] = inforaw[cols].fillna(0)
inforaw = inforaw.dropna()
inforaw = inforaw.reset_index()
inforaw = inforaw.sort_values(by=['tic', 'fyear'])

# Adjust for stock split
inforaw['csho'] = inforaw['csho'] * inforaw['ajex']
inforaw['mrk_eq'] = inforaw['cshom'] * inforaw['prccm']
inforaw

,index,datadate,cik,gvkey,tic,fyear,ib,spi,at,dvc,act,che,lct,dlc,ni,txp,revt,dp,csho,ceq,ivao,lt,ajex,dltt,ivst,pstk,cshom,prccm,mrk_eq
12342,14821,2009-06-30,0000770034,012837,3ADTI,2009.0,-5.586,0.000,5.747,0.0,3.990,1.320,8.328,7.241,-5.586,0.0,0.000,0.490,32.01500,-2.790,0.000,8.360,1.00,0.021,0.000,0.177,26198000.0,0.0600,1571880.0
23315,36778,2009-12-31,0000749254,024792,3AEMI,2009.0,-6.623,0.000,26.375,0.0,10.488,6.570,12.422,1.092,-6.623,0.0,29.422,1.610,22.08600,-11.200,0.000,37.575,1.00,18.406,0.800,0.000,21786000.0,0.3000,6535800.0
23316,36779,2010-12-31,0000749254,024792,3AEMI,2010.0,-1.305,-1.435,22.149,0.0,7.138,3.022,13.738,1.131,-1.305,0.0,27.493,1.801,33.14600,-9.766,0.000,31.915,1.00,17.590,0.601,0.000,22279000.0,0.2800,6238120.0
23312,36775,2009-12-31,0000879181,024668,3AHOM,2009.0,-13.098,0.000,239.297,0.0,66.048,22.194,264.160,229.120,-13.098,0.0,241.540,25.690,17.57300,-37.411,8.352,276.276,1.00,0.003,0.250,0.000,17573000.0,0.2500,4393250.0
341,435,2011-12-31,0000276750,001367,3AMBEQ,2011.0,-0.080,0.000,0.906,0.0,0.906,0.906,0.022,0.000,-0.080,0.0,0.000,0.000,4.66600,0.884,0.000,0.022,1.00,0.000,0.000,0.000,4666000.0,0.2550,1189830.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59277,106314,2018-02-28,0001489353,178186,ZZZOF,2017.0,-6.558,0.000,27.923,0.0,2.078,1.071,4.458,0.017,-6.558,0.0,0.000,0.010,0.99482,23.400,0.808,4.523,0.01,0.065,0.546,0.000,80651000.0,0.3399,27413274.9
59278,106315,2019-02-28,0001489353,178186,ZZZOF,2018.0,-26.442,-0.029,6.954,0.0,0.234,0.152,2.956,0.559,-26.442,0.0,0.000,0.015,1.22361,3.998,0.000,2.956,0.01,0.000,0.139,0.000,99482000.0,0.1692,16832354.4
59279,106316,2020-02-29,0001489353,178186,ZZZOF,2019.0,-7.685,0.002,1.720,0.0,0.086,0.000,3.726,0.841,-7.685,0.0,0.000,0.006,1.93957,-2.006,0.000,3.726,0.01,0.000,0.000,0.000,122527000.0,0.0140,1715378.0
59280,106317,2021-02-28,0001489353,178186,ZZZOF,2020.0,-0.521,0.000,0.034,0.0,0.023,0.013,1.195,0.157,0.610,0.0,0.000,0.005,1.93957,-1.161,0.000,1.195,0.01,0.000,0.000,0.000,193957000.0,0.0140,2715398.0


In [21]:
# Load analyst data for three years, training
startYear_an = 2010
endYear_an = 2018
dataquery_one = f"""
            SELECT  a.ticker, a. fpedats, a. statpers, a.actual, a.medest, a.fpi
            FROM ibes.statsum_epsus as a
            WHERE 
            fpi = '1'
            AND measure = 'EPS'
            AND EXTRACT(YEAR FROM a.statpers) BETWEEN '{startYear_an}' AND '{endYear_an}'
            AND EXTRACT(MONTH FROM a.statpers) = 6
        """
analystraw = db.raw_sql(dataquery_one)
linkdf = pd.read_excel("IBES_Compustat_Link_File.xls")
df_analyst_one_raw_train = pd.merge(analystraw, linkdf, how='inner', left_on=['ticker'], right_on=['TICKER'])
df_analyst_one_raw_train = df_analyst_one_raw_train.dropna()
df_analyst_one_raw_train = df_analyst_one_raw_train.reset_index()


startYear_an = 2010
endYear_an = 2017
dataquery_two = f"""
            SELECT  a.ticker, a. fpedats, a. statpers, a.actual, a.medest
            FROM ibes.statsum_epsus as a
            WHERE 
            fpi = '2'
            AND measure = 'EPS'
            AND EXTRACT(YEAR FROM a.statpers) BETWEEN '{startYear_an}' AND '{endYear_an}'
            AND EXTRACT(MONTH FROM a.statpers) = 6
        """
analystraw = db.raw_sql(dataquery_two)
df_analyst_two_raw_train = pd.merge(analystraw, linkdf, how='inner', left_on=['ticker'], right_on=['TICKER'])
df_analyst_two_raw_train = df_analyst_two_raw_train.dropna()
df_analyst_two_raw_train = df_analyst_two_raw_train.reset_index()

startYear_an = 2010
endYear_an = 2016
dataquery_three = f"""
            SELECT  a.ticker, a. fpedats, a. statpers, a.actual, a.medest
            FROM ibes.statsum_epsus as a
            WHERE 
            fpi = '3'
            AND measure = 'EPS'
            AND EXTRACT(YEAR FROM a.statpers) BETWEEN '{startYear_an}' AND '{endYear_an}'
            AND EXTRACT(MONTH FROM a.statpers) = 6
        """
analystraw = db.raw_sql(dataquery_three)
df_analyst_three_raw_train = pd.merge(analystraw, linkdf, how='inner', left_on=['ticker'], right_on=['TICKER'])
df_analyst_three_raw_train = df_analyst_three_raw_train.dropna()
df_analyst_three_raw_train = df_analyst_three_raw_train.reset_index()


In [122]:
df_analyst_one_raw_train

,index,ticker,fpedats,statpers,actual,medest,fpi,gvkey,TICKER
0,0,0000,2014-12-31,2014-06-19,1.21,0.58,1,192458,0000
1,2,0000,2015-12-31,2015-06-18,1.02,0.83,1,192458,0000
2,3,000R,2016-12-31,2016-06-16,0.19,0.16,1,19352,000R
3,4,000R,2017-12-31,2017-06-15,0.69,0.38,1,19352,000R
4,5,000R,2014-12-31,2014-06-19,-1.05,-1.35,1,19352,000R
...,...,...,...,...,...,...,...,...,...
32321,34789,ZXZX,2013-12-31,2013-06-20,0.74,1.38,1,187073,ZXZX
32322,34790,YOGA,2018-12-31,2018-06-14,-0.88,-0.83,1,32192,YOGA
32323,34793,XELA,2018-12-31,2018-06-14,-46.20,1.20,1,32074,XELA
32324,34795,ZLAB,2018-12-31,2018-06-14,-2.64,-1.03,1,32334,ZLAB


In [23]:
# Load analyst data for three years, testing
startYear_an = 2019
endYear_an = 2019
dataquery_one = f"""
            SELECT  a.ticker, a. fpedats, a. statpers, a.actual, a.medest, a.fpi
            FROM ibes.statsum_epsus as a
            WHERE 
            fpi = '1'
            AND measure = 'EPS'
            AND EXTRACT(YEAR FROM a.statpers) BETWEEN '{startYear_an}' AND '{endYear_an}'
            AND EXTRACT(MONTH FROM a.statpers) = 6
        """
analystraw = db.raw_sql(dataquery_one)
linkdf = pd.read_excel("IBES_Compustat_Link_File.xls")
df_analyst_raw_test_2019 = pd.merge(analystraw, linkdf, how='inner', left_on=['ticker'], right_on=['TICKER'])
df_analyst_raw_test_2019 = df_analyst_raw_test_2019.dropna()
df_analyst_raw_test_2019 = df_analyst_raw_test_2019.reset_index()

startYear_an = 2022
endYear_an = 2022
dataquery_two = f"""
            SELECT  a.ticker, a. fpedats, a. statpers, a.actual, a.medest
            FROM ibes.statsum_epsus as a
            WHERE 
            fpi = '1'
            AND measure = 'EPS'
            AND EXTRACT(YEAR FROM a.statpers) BETWEEN '{startYear_an}' AND '{endYear_an}'
            AND EXTRACT(MONTH FROM a.statpers) = 6
        """
analystraw = db.raw_sql(dataquery_two)
df_analyst_raw_test_2022 = pd.merge(analystraw, linkdf, how='inner', left_on=['ticker'], right_on=['TICKER'])
df_analyst_raw_test_2022 = df_analyst_raw_test_2022.dropna()
df_analyst_raw_test_2022 = df_analyst_raw_test_2022.reset_index()


In [25]:
pd.merge(inforaw, df_analyst_raw_test_2019, how='inner', left_on=['tic', 'datadate'], right_on=['ticker', 'fpedats'])


,index_x,datadate,cik,gvkey_x,tic,fyear,ib,spi,at,dvc,act,che,lct,dlc,ni,txp,revt,dp,csho,ceq,ivao,lt,ajex,dltt,ivst,pstk,cshom,prccm,mrk_eq,index_y,ticker,fpedats,statpers,actual,medest,fpi,gvkey_y,TICKER
0,27476,2019-12-31,0001158114,018564,AAOI,2019.0,-66.049,-11.030,466.825,0.000,192.804,67.028,91.355,40.663,-66.049,0.000,190.872,24.014,20.140,273.794,0.000,193.031,1.0,101.676,7.051,0.0,1.994000e+07,10.28,2.049832e+08,65,AAOI,2019-12-31,2019-06-20,-0.8600,-1.07,1,18564,AAOI
1,30725,2019-12-31,0000824142,021542,AAON,2019.0,53.711,0.000,371.424,16.645,187.549,44.373,56.028,0.000,53.711,0.000,469.333,22.766,52.079,290.140,0.597,0.000,1.0,0.000,17.576,0.0,5.211500e+07,50.18,2.615131e+09,1415,AAON,2019-12-31,2019-06-20,1.0200,1.20,1,21542,AAON
2,952,2019-09-30,0000320193,001690,AAPL,2019.0,55256.000,0.000,338516.000,14129.000,162819.000,100580.000,105718.000,16240.000,55256.000,0.000,260174.000,11300.000,17772.944,90488.000,105341.000,248028.000,4.0,91807.000,51736.000,0.0,4.601075e+09,197.92,9.106448e+11,513,AAPL,2019-09-30,2019-06-20,2.9725,2.86,1,1690,AAPL
3,21991,2019-12-31,0001551152,016101,ABBV,2019.0,7882.000,-4275.000,89115.000,6533.000,49519.000,39924.000,15585.000,3862.000,7882.000,0.000,33266.000,2017.000,1478.912,-8172.000,93.000,97287.000,1.0,63226.000,0.000,0.0,1.478365e+09,72.72,1.075067e+11,731,ABBV,2019-12-31,2019-06-20,8.9400,8.82,1,16101,ABBV
4,573,2019-10-31,0000771497,001410,ABM,2019.0,127.500,-17.600,3692.600,47.700,1275.400,58.500,902.400,57.200,127.400,3.500,6498.600,107.400,66.571,1542.000,0.000,2150.600,1.0,744.200,0.000,0.0,6.635800e+07,40.00,2.654320e+09,516,ABM,2019-10-31,2019-06-20,2.0500,2.03,1,1410,ABM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,51808,2019-07-31,0001713683,033037,ZS,2019.0,-28.655,-13.379,604.162,0.000,496.086,364.646,261.949,0.000,-28.655,0.000,302.836,11.306,127.253,308.558,0.000,295.604,1.0,0.000,286.162,0.0,1.256620e+08,76.64,9.630736e+09,3255,ZS,2019-07-31,2019-06-20,0.2200,0.17,1,33037,ZS
1446,44991,2019-12-31,0001677250,028423,ZTO,2019.0,814.899,100.308,6590.620,182.494,2886.435,2353.960,959.492,42.902,814.899,11.528,3175.347,181.612,781.947,5500.875,608.090,1075.270,1.0,72.447,1597.073,0.0,5.818200e+08,19.12,1.112440e+10,3774,ZTO,2019-12-31,2019-06-20,6.7500,6.47,1,28423,ZTO
1447,90805,2020-01-31,0001318008,162988,ZUMZ,2019.0,66.881,0.000,914.258,0.000,412.588,251.196,159.710,61.800,66.881,4.686,1034.129,25.134,25.828,466.086,1.711,448.172,1.0,284.717,198.768,0.0,2.574100e+07,26.10,6.718401e+08,3775,ZUMZ,2020-01-31,2019-06-20,2.6200,1.90,1,162988,ZUMZ
1448,53815,2020-01-31,0001423774,033232,ZUO,2019.0,-83.394,0.818,402.227,0.000,266.784,171.937,165.620,10.187,-83.394,0.432,274.757,11.276,114.482,164.659,0.000,237.568,1.0,68.401,117.662,0.0,8.647100e+07,15.32,1.324736e+09,3672,ZUO,2020-01-31,2019-06-20,-0.3400,-0.42,1,33232,ZUO


## ADV preparation

In [120]:
# A function taking care of all financial values calculation
def  calValues(db, company_list):
    print('Estimated waiting time 4mins, please wait')
    # Prepare dataset for nine models
    # HVZ one year
    df_HVZ_one = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'E_1'])
    # HVZ two year
    df_HVZ_two = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'E_2'])
    # HVZ three year
    df_HVZ_three = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'E_3'])

     # HVZ one year additional features
    df_HVZ_one_add = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover', 'medest', 'actual', 'E_lag1'])
    # HVZ two year
    df_HVZ_two_add = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'csho', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover', 'medest', 'actual', 'E_lag2'])
    # HVZ three year
    df_HVZ_three_add = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'csho', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover', 'medest', 'actual', 'E_lag3'])

    # EP one year
    df_EP_one = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'E_1'])
    # EP two year
    df_EP_two = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'E_2'])
    # EP three year
    df_EP_three = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'E_3'])

    # RI one year
    df_RI_one = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC', 'E_1'])
    # RI two year
    df_RI_two = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC', 'E_2'])
    # RI three year
    df_RI_three = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC', 'E_3'])

    X_pred_HVZs = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E', 'NegE', 'AC'])
    X_pred_HVZs_add_2019 = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover', 'medest', 'tic', 'cik', 'fyear', 'mrk_eq', 'E_lag1'])
    X_pred_HVZs_add_2022 = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover', 'medest', 'tic', 'cik', 'fyear', 'mrk_eq', 'E_lag1'])
    X_pred_EPs = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E'])
    X_pred_RIs = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC'])

    Y_pred_one = pd.DataFrame(columns=['tic','E_deflat'])
    Y_pred_two = pd.DataFrame(columns=['tic','E_deflat'])
    Y_pred_three = pd.DataFrame(columns=['tic','E_deflat'])
    count = 0
    # Consistent values calculation
    # Main model
    # HVZ model
    inforaw['E'] = inforaw['ib'] - inforaw['spi']
    inforaw['DD'] = (inforaw['dvc'] > 0).astype(int)
    inforaw['NegE'] = (inforaw['E'] < 0).astype(int)
    inforaw['AC'] = (inforaw['act'] - inforaw['che']).diff() - (inforaw['lct'] - inforaw['dlc'] - inforaw['txp']).diff() - inforaw['dp']

    # EP model
    inforaw['E_deflat'] = inforaw['E'] / inforaw['csho']
    inforaw['NegE_E'] = inforaw['E_deflat'] * inforaw['NegE']

    # RI model
    inforaw['B'] = inforaw['ceq'] / inforaw['csho']
    inforaw['WC'] = (inforaw['act'] - inforaw['che']) - (inforaw['lct'] - inforaw['dlc'])
    inforaw['NCO'] = (inforaw['at'] - inforaw['act'] - inforaw['ivao']) - (inforaw['lt'] - inforaw['lct'] - inforaw['dltt'])
    inforaw['FIN'] = ((inforaw['ivst'] - inforaw['ivao']) - (inforaw['dltt'] - inforaw['dlc'] - inforaw['pstk'])) 
    inforaw['TACC'] = (inforaw['WC'] + inforaw['NCO'] + inforaw['FIN']) / inforaw['csho']

    # Prediction
    inforaw['E_lag1'] = inforaw.groupby('tic')['E'].shift(-1)
    inforaw['E_lag2'] = inforaw.groupby('tic')['E'].shift(-2)
    inforaw['E_lag3'] = inforaw.groupby('tic')['E'].shift(-3)
    # Additional Features
    # Calculate Shareholder's Equity
    inforaw['shareholders_equity'] = inforaw['at'] - inforaw['lt']

    # Calculate current ratio
    inforaw['current_ratio'] = inforaw['act'] / inforaw['lct']

    # Calculate debt to equity
    inforaw['debt_to_equity'] = inforaw['lt'] / inforaw['shareholders_equity']

    # Calculate return on equity (ROE)
    inforaw['roe'] = inforaw['ni'] / inforaw['shareholders_equity']

    # Calculate Asset Turnover
    inforaw['asset_turnover'] = inforaw['revt'] / inforaw['at']

    df_analyst_one_train = pd.merge(inforaw, df_analyst_one_raw_train, how='inner', left_on=['tic', 'datadate'], right_on=['ticker', 'fpedats'])
    df_analyst_two_train = pd.merge(inforaw, df_analyst_two_raw_train, how='inner', left_on=['tic', 'datadate'], right_on=['ticker', 'fpedats'])
    df_analyst_three_train = pd.merge(inforaw, df_analyst_three_raw_train, how='inner', left_on=['tic', 'datadate'], right_on=['ticker', 'fpedats'])
    df_analyst_test_2019 = pd.merge(inforaw, df_analyst_raw_test_2019, how='inner', left_on=['tic', 'datadate'], right_on=['ticker', 'fpedats'])
    df_analyst_test_2022 = pd.merge(inforaw, df_analyst_raw_test_2022, how='inner', left_on=['tic', 'datadate'], right_on=['ticker', 'fpedats'])


    X_pred_HVZ_add = df_analyst_test_2019[['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover', 'medest', 'tic', 'cik', 'fyear', 'mrk_eq','E_lag1']].reset_index(drop=True)
    X_pred_HVZs_add_2019 = pd.concat([X_pred_HVZs_add_2019, X_pred_HVZ_add])
    X_pred_HVZs_add_2019 = X_pred_HVZs_add_2019.dropna()
    X_pred_HVZ_add = df_analyst_test_2022[['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover', 'medest', 'tic', 'cik', 'fyear', 'mrk_eq','E_lag1']].reset_index(drop=True)
    X_pred_HVZs_add_2022 = pd.concat([X_pred_HVZs_add_2022, X_pred_HVZ_add])
    X_pred_HVZs_add_2022 = X_pred_HVZs_add_2022.dropna()

    
    for cik in company_list['cik'].astype(str):
        count += 1
        # accrualsinfo = db.raw_sql(dataquery)
        infocomp = inforaw.loc[(inforaw['cik'] == cik)].reset_index(drop=True)
        infocomp = infocomp.sort_values(by='fyear')

        df_analyst_one = df_analyst_one_train.loc[(df_analyst_one_train['cik'] == cik)].reset_index(drop=True)
        df_analyst_one = df_analyst_one.sort_values(by='fyear')
        df_analyst_two = df_analyst_two_train.loc[(df_analyst_two_train['cik'] == cik)].reset_index(drop=True)
        df_analyst_two = df_analyst_two.sort_values(by='fyear')
        df_analyst_three = df_analyst_three_train.loc[(df_analyst_three_train['cik'] == cik)].reset_index(drop=True)
        df_analyst_three = df_analyst_three.sort_values(by='fyear')

        # Compile dataset
        # Drop year differnece axillary 
        infocomp = infocomp[infocomp['fyear'] != 2009]
        df_analyst_one = df_analyst_one.sort_values(by='fyear')
        df_analyst_two = df_analyst_two.sort_values(by='fyear')
        df_analyst_three = df_analyst_three.sort_values(by='fyear')
        # Year range of training
        infocomptrain = infocomp[(infocomp['fyear'] >= 2010) & (infocomp['fyear'] <= 2019)].reset_index(drop=True)
        # Prediction data
        X_pred_2019 = infocomptrain.loc[infocomptrain['fyear'] == 2019].reset_index(drop=True)
        X_pred_2019 = X_pred_2019.dropna()
        if X_pred_2019.empty is not None: 
            # Yearly data
            Y_pred_2020 = infocomp[(infocomp['fyear'] == 2020)].reset_index(drop=True)
            Y_pred_2020 = Y_pred_2020[['tic','E_deflat']] #  !!!!!
            Y_pred_one = Y_pred_one.append(Y_pred_2020, ignore_index=True)

            Y_pred_2021 = infocomp[(infocomp['fyear'] == 2021)].reset_index(drop=True)
            Y_pred_2021 = Y_pred_2021[['tic','E_deflat']]
            Y_pred_two = Y_pred_two.append(Y_pred_2021, ignore_index=True)

            Y_pred_2022 = infocomp[(infocomp['fyear'] == 2022)].reset_index(drop=True)
            Y_pred_2022 = Y_pred_2022[['tic','E_deflat']]
            Y_pred_three = Y_pred_three.append(Y_pred_2022, ignore_index=True)

        ## Calculate HVZ Prediction, One year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2018, ['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho']].reset_index(drop=True)
        X_add = df_analyst_one.loc[df_analyst_one['fyear'] <= 2018, ['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover', 'medest', 'actual', 'tic', 'cik', 'fyear']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2011, ['E']].reset_index(drop=True)
        y_add = df_analyst_one.loc[df_analyst_one['fyear'] >= 2011, ['E_lag1']].reset_index(drop=True)
        
        X_pred_HVZ = X_pred_2019[['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho', 'gvkey', 'tic']].reset_index(drop=True)
        X_pred_HVZs = X_pred_HVZs.append(X_pred_HVZ, ignore_index=True)

        y = y.rename(columns={"E": "E_1"})
        HVZ_one = pd.concat([X, y], axis=1)
        df_HVZ_one = df_HVZ_one.append(HVZ_one, ignore_index=True)
        df_HVZ_one = df_HVZ_one.dropna()
        HVZ_one_add = pd.concat([X_add, y_add], axis=1)
        df_HVZ_one_add = df_HVZ_one_add.append(HVZ_one_add, ignore_index=True)
        df_HVZ_one_add = df_HVZ_one_add.dropna()


        ## Calculate HVZ Prediction, Two year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2017, ['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho']].reset_index(drop=True)
        X_add = df_analyst_two.loc[df_analyst_two['fyear'] <= 2017, ['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover', 'medest', 'actual']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2012, ['E']].reset_index(drop=True)
        y_add = df_analyst_two.loc[df_analyst_two['fyear'] >= 2011, ['E_lag2']].reset_index(drop=True)
        y = y.rename(columns={"E": "E_2"})
        HVZ_two = pd.concat([X, y], axis=1)
        df_HVZ_two = df_HVZ_two.append(HVZ_two, ignore_index=True)
        df_HVZ_two = df_HVZ_two.dropna()
        HVZ_two_add = pd.concat([X_add, y_add], axis=1)
        df_HVZ_two_add = df_HVZ_two_add.append(HVZ_two_add, ignore_index=True)
        df_HVZ_two_add = df_HVZ_two_add.dropna()

        ## Calculate HVZ Prediction, Three year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2016, ['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho']].reset_index(drop=True)
        X_add = df_analyst_three.loc[df_analyst_three['fyear'] <= 2016, ['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover', 'medest', 'actual']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2013, ['E']].reset_index(drop=True)
        y_add = df_analyst_three.loc[df_analyst_three['fyear'] >= 2011, ['E_lag3']].reset_index(drop=True)
        y = y.rename(columns={"E": "E_3"})
        HVZ_three = pd.concat([X, y], axis=1)
        df_HVZ_three = df_HVZ_three.append(HVZ_three, ignore_index=True)
        df_HVZ_three = df_HVZ_three.dropna()
        HVZ_three_add = pd.concat([X_add, y_add], axis=1)
        df_HVZ_three_add = df_HVZ_three_add.append(HVZ_three_add, ignore_index=True)
        df_HVZ_three_add = df_HVZ_three_add.dropna()

        ## Calculate EP Prediction, One year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2018, ['E_deflat', 'NegE', 'NegE_E']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2011, ['E_deflat']].reset_index(drop=True)

        X_pred_EP = X_pred_2019[['E_deflat', 'NegE', 'NegE_E', 'gvkey', 'tic']].reset_index(drop=True)
        X_pred_EPs = X_pred_EPs.append(X_pred_EP, ignore_index=True)

        y = y.rename(columns={"E_deflat": "E_1"})
        EP_one = pd.concat([X, y], axis=1)
        df_EP_one = df_EP_one.append(EP_one, ignore_index=True)
        df_EP_one = df_EP_one.dropna()

        ## Calculate EP Prediction, Two year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2017, ['E_deflat', 'NegE', 'NegE_E']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2012, ['E_deflat']].reset_index(drop=True)
        y = y.rename(columns={"E_deflat": "E_2"})
        EP_two = pd.concat([X, y], axis=1)
        df_EP_two = df_EP_two.append(EP_two, ignore_index=True)
        df_EP_two = df_EP_two.dropna()

        ## Calculate EP Prediction, Three year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2016, ['E_deflat', 'NegE', 'NegE_E']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2013, ['E_deflat']].reset_index(drop=True)
        y = y.rename(columns={"E_deflat": "E_3"})
        EP_three = pd.concat([X, y], axis=1)
        df_EP_three = df_EP_three.append(EP_three, ignore_index=True)
        df_EP_three = df_EP_three.dropna()

        ## Calculate RI Prediction, One year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2018, ['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2011, ['E_deflat']].reset_index(drop=True)
        
        X_pred_RI = X_pred_2019[['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC', 'gvkey', 'tic']].reset_index(drop=True)
        X_pred_RIs = X_pred_RIs.append(X_pred_RI, ignore_index=True)
        
        y = y.rename(columns={"E_deflat": "E_1"})
        RI_one = pd.concat([X, y], axis=1)
        df_RI_one = df_RI_one.append(RI_one, ignore_index=True)
        df_RI_one = df_RI_one.dropna()

        ## Calculate RI Prediction, Two year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2017, ['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2012, ['E_deflat']].reset_index(drop=True)
        y = y.rename(columns={"E_deflat": "E_2"})
        RI_two = pd.concat([X, y], axis=1)
        df_RI_two = df_RI_two.append(RI_two, ignore_index=True)
        df_RI_two = df_RI_two.dropna()

        ## Calculate RI Prediction, Three year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2016, ['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2013, ['E_deflat']].reset_index(drop=True)
        y = y.rename(columns={"E_deflat": "E_3"})
        RI_three = pd.concat([X, y], axis=1)
        df_RI_three = df_RI_three.append(RI_three, ignore_index=True)
        df_RI_three = df_RI_three.dropna()
        
    return df_HVZ_one, df_HVZ_two, df_HVZ_three, \
        df_EP_one, df_EP_two, df_EP_three, \
            df_RI_one, df_RI_two,df_RI_three, \
                X_pred_HVZs, X_pred_EPs, X_pred_RIs, Y_pred_one, Y_pred_two, Y_pred_three,\
                      df_HVZ_one_add, df_HVZ_two_add, df_HVZ_three_add, \
                        X_pred_HVZs_add_2019,X_pred_HVZs_add_2022
        # infocomptrain


In [88]:
#Compile companylist for companies that EXISTED in EVERY year between startyeartrain to endyeartest to not Nan the value
year = startYear
query_comp = (
        f"""
        SELECT DISTINCT cik
        FROM comp.funda
        WHERE fyear = {year}
        AND consol = 'C'
        AND popsrc = 'D'
        AND indfmt = 'INDL'
        AND datafmt = 'STD'  
        """) 
company_list = db.raw_sql(query_comp) 
for year in np.arange(startYear,endYear):
    query_comp = (
        f"""
        SELECT DISTINCT cik
        FROM comp.funda
        WHERE fyear = {year}
        AND consol = 'C'
        AND popsrc = 'D'
        AND indfmt = 'INDL'
        AND datafmt = 'STD'  
        """) 
    comp_list = db.raw_sql(query_comp) 
    company_list = pd.merge(comp_list,company_list, on= 'cik', how='inner')
company_list = company_list.dropna().reset_index(drop=True)

# Company list obtained, Get company tic for reviewing convenience
query_comp = (
        f"""
        SELECT DISTINCT cik, MIN (tic) AS tic
        FROM comp.funda
        WHERE fyear BETWEEN '{startYear}' AND '{endYear}'
        AND consol = 'C'
        AND popsrc = 'D'
        AND indfmt = 'INDL'
        AND datafmt = 'STD'  
        group by cik
        """) 
company_tic_list = db.raw_sql(query_comp) 
company_list = pd.merge(company_list, company_tic_list, left_on = 'cik', right_on = 'cik', how='left')
# Make sure we have the tic exist in linkdf so we can find analyst data
company_list = pd.merge(company_list, linkdf, left_on = 'tic', right_on = 'TICKER', how='inner')

print(company_list)


#!!!!!!
# company_list = company_list.head(100)

             cik   tic   gvkey TICKER
0     0001119774  CGEN    3172   CGEN
1     0001115837   MBT  137433    MBT
2     0000075362  PCAR    8253   PCAR
3     0001395942   KAR  183581    KAR
4     0000871763   MAN    7007    MAN
...          ...   ...     ...    ...
1854  0001289848  HURN  161997   HURN
1855  0001084048    ZD   66716     ZD
1856  0001345105   CPA   21381    CPA
1857  0000882361  APTO   15490   APTO
1858  0001046025  HFWA   66285   HFWA

[1859 rows x 4 columns]


In [89]:
# Data extraction
df_HVZ_one, df_HVZ_two, df_HVZ_three, \
        df_EP_one, df_EP_two, df_EP_three, \
            df_RI_one, df_RI_two,df_RI_three, \
                X_pred_HVZs, X_pred_EPs, X_pred_RIs, Y_pred_one, Y_pred_two, Y_pred_three,\
                      df_HVZ_one_add, df_HVZ_two_add, df_HVZ_three_add, \
                        X_pred_HVZs_add_2019,X_pred_HVZs_add_2022 = calValues(db, company_list)

Estimated waiting time 4mins, please wait
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266


In [156]:
# How to use the object
# 1. Load the dictionary as dataframe
lm_word_list_df = pd.read_csv('Loughran-McDonald_MasterDictionary_1993-2021.csv')
# 2. Create a TextSentimentAnalyser object
text_analyzer = TextSentimentAnalyser(lm_word_list_df)
df_HVZ_one_add['sentiment'] = None
for index,row in df_HVZ_one_add.iterrows():    
    # 3. Pass in company ticker and year to calculate sentiment. If company does not have 10k
    # that year it will return null
    print(int(row['cik']))
    sentiment = text_analyzer.get_sentiment(row['cik'], int(row['fyear']))
    df_HVZ_one_add.at[index, 'sentiment'] = sentiment
    print(sentiment)


1115837
No 10k found for 0001115837 in 2010.
NoSentiment
1115837
No 10k found for 0001115837 in 2011.
NoSentiment
1115837
No 10k found for 0001115837 in 2012.
NoSentiment
1115837
No 10k found for 0001115837 in 2012.
NoSentiment
1115837
No 10k found for 0001115837 in 2013.
NoSentiment
1115837
No 10k found for 0001115837 in 2014.
NoSentiment
1115837
No 10k found for 0001115837 in 2015.
NoSentiment
1115837
No 10k found for 0001115837 in 2016.
NoSentiment
1115837
No 10k found for 0001115837 in 2017.
NoSentiment
1395942
Negative
1395942
Negative
1395942
Negative
1395942
Negative
1395942
Negative
1395942
Negative
1395942
Uncertainty
1395942
Negative
949157
Negative
949157
Negative
949157
Negative
949157
Negative
949157
Negative
949157
Uncertainty
949157
Negative
949157


KeyboardInterrupt: 

In [ ]:
# How to use the object
# 1. Load the dictionary as dataframe
lm_word_list_df = pd.read_csv('Loughran-McDonald_MasterDictionary_1993-2021.csv')
# 2. Create a TextSentimentAnalyser object
text_analyzer = TextSentimentAnalyser(lm_word_list_df)
X_pred_HVZs_add_2019['sentiment'] = None
for index,row in X_pred_HVZs_add_2019.iterrows():    
    # 3. Pass in company ticker and year to calculate sentiment. If company does not have 10k
    # that year it will return null
    print(int(row['cik']))
    sentiment = text_analyzer.get_sentiment(row['cik'], int(row['fyear']))
    X_pred_HVZs_add_2019.at[index, 'sentiment'] = sentiment
    print(sentiment)

In [ ]:
# How to use the object
# 1. Load the dictionary as dataframe
lm_word_list_df = pd.read_csv('Loughran-McDonald_MasterDictionary_1993-2021.csv')
# 2. Create a TextSentimentAnalyser object
text_analyzer = TextSentimentAnalyser(lm_word_list_df)
X_pred_HVZs_add_2022['sentiment'] = None
for index,row in X_pred_HVZs_add_2022.iterrows():    
    # 3. Pass in company ticker and year to calculate sentiment. If company does not have 10k
    # that year it will return null
    print(int(row['cik']))
    sentiment = text_analyzer.get_sentiment(row['cik'], int(row['fyear']))
    X_pred_HVZs_add_2022.at[index, 'sentiment'] = sentiment
    print(sentiment)

## Build Model

### Q1 model

In [126]:
df_HVZ_one_add

,at,dvc,DD,E,NegE,AC,csho,current_ratio,debt_to_equity,roe,asset_turnover,medest,actual,E_lag1,tic,cik,fyear,sentiment
0,14478.042,953.192,1,1431.331,0,-1932.03,958.435,0.987786,2.411618,0.325333,0.780025,1.95,1.55,2036.445,MBT,0001115837,2010.0,NoSentiment
1,15318.229,1026.747,1,1443.944,0,-2300.028,994.458,1.076529,3.195366,0.395468,0.804185,1.87,1.26,2322.755,MBT,0001115837,2011.0,NoSentiment
2,14979.848,881.945,1,2036.445,0,-2410.399,994.46,0.814441,2.912475,0.263085,0.830159,1.76,51.04,2322.755,MBT,0001115837,2012.0,NoSentiment
3,14979.848,881.945,1,2036.445,0,-2410.399,994.46,0.814441,2.912475,0.263085,0.830159,1.76,0.822,889.503,MBT,0001115837,2012.0,NoSentiment
4,14818.192,1203.068,1,2322.755,0,-1752.056,994.416,1.145432,2.053898,0.502179,0.820645,1.97,80.28,747.79,MBT,0001115837,2013.0,NoSentiment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6302,885.6,0.0,0,61.349,0,-21.042,23.253,1.571737,0.670111,0.125283,0.889554,2.61,2.96,56.948,HURN,0001289848,2013.0,None
6303,1155.914,0.0,0,81.899,0,-17.157,22.879,2.756378,0.92449,0.131613,0.769267,3.23,3.42,47.082,HURN,0001289848,2014.0,None
6304,1164.16,0.0,0,56.948,0,-83.853,22.526,1.746973,0.784632,0.090525,0.660582,3.81,2.99,90.714,HURN,0001289848,2015.0,None
6305,1153.215,0.0,0,47.082,0,-59.083,21.718,1.326395,0.779562,0.058048,0.691965,3.35,3.21,21.289,HURN,0001289848,2016.0,None


In [131]:
# Prepare dataset for nine models
y_pred_dict = {}
y_pred_adjusted_dict = {}

# Define a dictionary to store the models
models = {}

# HVZ one year
data = {'HVZ_one': df_HVZ_one, 'HVZ_two': df_HVZ_two, 'HVZ_three': df_HVZ_three,
        'EP_one': df_EP_one, 'EP_two': df_EP_two, 'EP_three': df_EP_three,
        'RI_one': df_RI_one, 'RI_two': df_RI_two, 'RI_three': df_RI_three}

for name, df in data.items(): 
    print(name)
    if 'HVZ' in name:
        X = df[['at', 'dvc', 'DD', 'E', 'NegE', 'AC']]
    elif 'EP' in name:
        X = df[['E_deflat', 'NegE', 'NegE_E']]
    elif 'RI' in name:
        X = df[['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC']]
    
    if 'one' in name:
        y = df['E_1']
    elif 'two' in name:
        y = df['E_2']
    elif 'three' in name:
        y = df['E_3']

    X = add_constant(X)
    
    # Define and fit the model
    model = LinearRegression()
    model.fit(X, y)

    # Store the model in the dictionary
    models[name] = model

    # Predict using the model
    y_pred = model.predict(X)
    y_pred_dict[name] = y_pred

    # print('Bias:')
    # print(y - y_pred) 
    # Calculate R-squared
    r_squared = r2_score(y, y_pred)

    # Calculate the adjusted R-squared score
    n = len(X)
    p = len(model.coef_)
    adj_r2 = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)
    print("Adjusted R-squared Score: ", adj_r2)

    # coefficients = pd.DataFrame({"Variable": X.columns, "Coefficient": model.coef_})
    # print(coefficients)

    if 'HVZ' in name:
        y_pred_adjusted = y_pred / df['csho']
        y_pred_adjusted_dict[name] = y_pred_adjusted


HVZ_one
Adjusted R-squared Score:  0.8406585034002997
HVZ_two
Adjusted R-squared Score:  0.765161741525018
HVZ_three
Adjusted R-squared Score:  0.7207776428117876
EP_one
Adjusted R-squared Score:  0.6517038844402605
EP_two
Adjusted R-squared Score:  0.37627104332115535
EP_three
Adjusted R-squared Score:  0.09523489577762567
RI_one
Adjusted R-squared Score:  0.6586419773745986
RI_two
Adjusted R-squared Score:  0.5278529905972255
RI_three
Adjusted R-squared Score:  0.14284848752340384


In [113]:
# Compare to analyst 
# One, two and three year models
models = {'HVZ_one': models['HVZ_one'], 'EP_one': models['EP_one'], 'RI_one': models['RI_one'],
          'HVZ_two': models['HVZ_two'], 'EP_two': models['EP_two'], 'RI_two': models['RI_two'],
          'HVZ_three': models['HVZ_three'], 'EP_three': models['EP_three'], 'RI_three': models['RI_three']}


for model_name, model in models.items():
    print(model_name)
    if 'HVZ' in model_name:
        X = X_pred_HVZs[['at', 'dvc', 'DD', 'E', 'NegE', 'AC']]
    elif 'EP' in model_name:
        X = X_pred_EPs[['E_deflat', 'NegE', 'NegE_E']]
    elif 'RI' in model_name:
        X = X_pred_RIs[['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC']]
    
    X = add_constant(X)
    
    if 'one' in model_name:
        y = Y_pred_one.copy()
        print(1)
    elif 'two' in model_name:
        y = Y_pred_two.copy()
        print(2)
    elif 'three' in model_name:
        y = Y_pred_three.copy()
        print(3)
    y_pred = model.predict(X)
    y_pred_df = pd.DataFrame(y_pred, columns=['y_pred'])
    # y_pred_df = y_pred_df.
    if 'HVZ' in model_name:
        # y_pred_adjusted = y_pred_df['y_pred'] / X_pred_HVZs['csho']
        y_pred_df['y_pred'] = y_pred_df['y_pred'] / X_pred_HVZs['csho']
        # y_pred_adjusted_dict[name] = y_pred_adjusted
        # print(y_pred)
    y_pred_df = pd.concat([X_pred_HVZs['tic'], y_pred_df], axis=1)
    # y_pred_df = pd.concat([y_pred_df, y], axis=1)
    y_pred_df = pd.merge(y_pred_df,y, on= 'tic', how='inner')

    y_pred_df['bias'] = (y_pred_df['E_deflat'] - y_pred_df['y_pred'])

    print(y_pred_df)



HVZ_one
1
       tic    y_pred  E_deflat       bias
0     CGEN  0.239526 -0.354917  -0.594443
1      KAR  1.594511  0.135698  -1.458813
2      MAN  8.331016  4.018644  -4.312372
3     CENX -0.307839 -1.355823  -1.047984
4      JKS  3.749204  0.748138  -3.001066
...    ...       ...       ...        ...
1157    NI  1.768977  1.769706   0.000729
1158  HURN  2.848778   2.63836  -0.210417
1159    ZD  5.030404  4.332717  -0.697687
1160   CPA  7.953569 -8.301046 -16.254615
1161  APTO  0.225532 -0.621476  -0.847008

[1162 rows x 4 columns]
EP_one
1
       tic     y_pred  E_deflat       bias
0     CGEN -37.697544 -0.354917  37.342628
1      KAR  10.495114  0.135698 -10.359416
2      MAN -31.047628  4.018644  35.066273
3     CENX -37.963908 -1.355823  36.608085
4      JKS  -0.546724  0.748138   1.294862
...    ...        ...       ...        ...
1157    NI   8.101657  1.769706  -6.331951
1158  HURN   4.893513   2.63836  -2.255153
1159    ZD -14.556457  4.332717  18.889174
1160   CPA -31.665538 

In [ ]:
y_pred_df['E_deflat'].describe(include='all')

count     1158.000000
unique    1158.000000
top         -0.388964
freq         1.000000
Name: E_deflat, dtype: float64

In [ ]:
y_pred_df['E_deflat'].max()

127.03147547840634

### Q2 Model, with additional features


In [155]:
# Prepare dataset for nine models
y_pred_dict = {}
y_pred_adjusted_dict = {}

# Define a dictionary to store the models
models = {}

# HVZ one year
data = {'HVZ_one': df_HVZ_one_add}

for name, df in data.items(): 
    print(name)
    X = df[['at', 'dvc', 'DD', 'E','NegE', 'AC', 'current_ratio', 'debt_to_equity', 'asset_turnover', 'medest']]
    y = df['E_lag1']

    X = add_constant(X)
    
    # Define and fit the model
    model = LinearRegression()
    model.fit(X, y)

    # Store the model in the dictionary
    models[name] = model

    # Predict using the model
    y_pred = model.predict(X)
    y_pred_dict[name] = y_pred

    # print('Bias:')
    # print(y - y_pred) 
    # Calculate R-squared
    r_squared = r2_score(y, y_pred)

    # Calculate the adjusted R-squared score
    n = len(X)
    p = len(model.coef_)
    adj_r2 = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)
    print("Adjusted R-squared Score: ", adj_r2)

    # coefficients = pd.DataFrame({"Variable": X.columns, "Coefficient": model.coef_})
    # print(coefficients)

    if 'HVZ' in name:
        y_pred_adjusted = y_pred / df['csho']
        y_pred_adjusted_dict[name] = y_pred_adjusted


HVZ_one
Adjusted R-squared Score:  0.8112590920426386


In [95]:
X_pred_HVZs_add_2019

,at,dvc,DD,E,NegE,AC,csho,current_ratio,debt_to_equity,roe,asset_turnover,medest,tic,cik,fyear,E_lag1
0,466.825,0.0,0,-55.019,1,-36.0,20.14,2.110492,0.705023,-0.241236,0.408873,-1.07,AAOI,0001158114,2019.0,-50.677
1,371.424,16.645,1,53.711,0,-26.414,52.079,3.347416,0.0,0.144608,1.263604,1.2,AAON,0000824142,2019.0,72.609
2,338516.0,14129.0,1,55256.0,0,-7459.0,17772.944,1.540126,2.741004,0.610645,0.768572,2.86,AAPL,0000320193,2019.0,57411.0
3,89115.0,6533.0,1,12157.0,0,-1098.0,1478.912,3.17735,-11.904919,-0.964513,0.373293,8.82,ABBV,0001551152,2019.0,15172.0
4,3692.6,47.7,1,145.1,0,-111.8,66.571,1.413342,1.394682,0.08262,1.759898,2.03,ABM,0000771497,2019.0,194.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,604.162,0.0,0,-15.276,1,-47.608,127.253,1.893827,0.958018,-0.092867,0.50125,0.17,ZS,0001713683,2019.0,-94.914
1446,6590.62,182.494,1,714.591,0,-303.463,781.947,3.008295,0.19496,0.147751,0.481798,6.47,ZTO,0001677250,2019.0,656.17
1447,914.258,0.0,0,66.881,0,-31.516,25.828,2.583357,0.961565,0.143495,1.131113,1.9,ZUMZ,0001318008,2019.0,81.03
1448,402.227,0.0,0,-84.212,1,-12.832,114.482,1.61082,1.442788,-0.506465,0.683089,-0.42,ZUO,0001423774,2019.0,-69.922


In [108]:
# using 2019 to predict 2020
# One, two and three year models
models = {'HVZ_one': models['HVZ_one'],
          'HVZ_two': models['HVZ_two'], 
          'HVZ_three': models['HVZ_three']}


for model_name, model in models.items():
    print(model_name)

    X = X_pred_HVZs_add_2019[['at', 'dvc', 'DD', 'E','NegE', 'AC', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover', 'medest']]
    X = add_constant(X)
    
    y = X_pred_HVZs_add_2019['E_lag1'] / X_pred_HVZs_add_2019['csho']
    y_pred = model.predict(X)
    y_pred_df = pd.DataFrame(y_pred, columns=['y_pred'])

    # y_pred_df = y_pred_df.
    
    y_pred_df['y_pred'] = y_pred_df['y_pred'] / X_pred_HVZs_add_2019['csho']

    y_pred_df = pd.concat([X_pred_HVZs_add_2019['tic'], y_pred_df], axis=1)
    
    print(y_pred_df)

    # # y_pred_df = pd.concat([y_pred_df, y], axis=1)
    # y_pred_df = pd.concat([y_pred_df, y], axis=1)
    # # print(y_pred_df.head(20))
    # y_pred_df['accuracy'] = np.absolute((y_pred_df[0] - y_pred_df['y_pred'])/y_pred_df[0])

    # print(y_pred_df.sort_values('accuracy', ascending=True))



HVZ_one
       tic    y_pred
0     AAOI  3.895654
1     AAON  1.404141
2     AAPL  2.905509
3     ABBV  8.755333
4      ABM  2.918559
...    ...       ...
1343   NaN       NaN
1367   NaN       NaN
1370   NaN       NaN
1377   NaN       NaN
1378   NaN       NaN

[1447 rows x 2 columns]
HVZ_two
       tic    y_pred
0     AAOI  4.729229
1     AAON  1.376813
2     AAPL  2.883354
3     ABBV  8.771579
4      ABM  2.908934
...    ...       ...
1343   NaN       NaN
1367   NaN       NaN
1370   NaN       NaN
1377   NaN       NaN
1378   NaN       NaN

[1447 rows x 2 columns]
HVZ_three
       tic    y_pred
0     AAOI   8.53435
1     AAON  1.856218
2     AAPL  2.964046
3     ABBV  8.993512
4      ABM  3.413394
...    ...       ...
1343   NaN       NaN
1367   NaN       NaN
1370   NaN       NaN
1377   NaN       NaN
1378   NaN       NaN

[1447 rows x 2 columns]


In [107]:
X_pred_HVZs_add_2019['tic']

0       AAOI
1       AAON
2       AAPL
3       ABBV
4        ABM
        ... 
1445      ZS
1446     ZTO
1447    ZUMZ
1448     ZUO
1449    ZYNE
Name: tic, Length: 1395, dtype: object

In [93]:
# using 2022 to predict 2023
models = {'HVZ_one': models['HVZ_one'],
          'HVZ_two': models['HVZ_two'], 
          'HVZ_three': models['HVZ_three']}


for model_name, model in models.items():
    print(model_name)

    X = X_pred_HVZs_add_2022[['at', 'dvc', 'DD', 'E','NegE', 'AC', 'current_ratio', 'debt_to_equity', 'roe', 'asset_turnover', 'medest']]
    
    X = add_constant(X)
    
    y = X_pred_HVZs_add_2022['E_lag1'] / X_pred_HVZs_add_2022['csho']
    y_pred = model.predict(X)
    print(y_pred)
    y_pred_df = pd.DataFrame(y_pred, columns=['y_pred'])
    print(y_pred_df)
    y_pred_df['y_pred'] = y_pred_df['y_pred'] / ((X_pred_HVZs_add_2022['csho']).astype(float))
    y_pred_df = pd.concat([X_pred_HVZs['tic'], y_pred_df], axis=1)
    # y_pred_df = pd.concat([y_pred_df, y], axis=1)
    y_pred_df = pd.concat([y_pred_df, y], axis=1)
    # print(y_pred_df.head(20))
    y_pred_df['accuracy'] = np.absolute((y_pred_df[0] - y_pred_df['y_pred'])/y_pred_df[0])

    print(y_pred_df.sort_values('accuracy', ascending=True))


# Print out E and E_forcast for percentage

HVZ_one
[  90.2794548    69.86070108  757.59315101   83.71121257 -131.17892538]
       y_pred
0   90.279455
1   69.860701
2  757.593151
3   83.711213
4 -131.178925
       tic  y_pred    0 accuracy
0     CGEN     NaN  NaN      NaN
1      KAR     NaN  NaN      NaN
2      MAN     NaN  NaN      NaN
3     CENX     NaN  NaN      NaN
4      JKS     NaN  NaN      NaN
...    ...     ...  ...      ...
1157    NI     NaN  NaN      NaN
1158  HURN     NaN  NaN      NaN
1159    ZD     NaN  NaN      NaN
1160   CPA     NaN  NaN      NaN
1161  APTO     NaN  NaN      NaN

[1162 rows x 4 columns]
HVZ_two
[ 109.99902767   85.02456893  685.83884998   58.99617138 -115.26643803]
       y_pred
0  109.999028
1   85.024569
2  685.838850
3   58.996171
4 -115.266438
       tic  y_pred    0 accuracy
0     CGEN     NaN  NaN      NaN
1      KAR     NaN  NaN      NaN
2      MAN     NaN  NaN      NaN
3     CENX     NaN  NaN      NaN
4      JKS     NaN  NaN      NaN
...    ...     ...  ...      ...
1157    NI     NaN  

In [86]:
y_pred_df['y_pred']

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
85    NaN
172   NaN
311   NaN
576   NaN
635   NaN
Name: y_pred, Length: 75, dtype: float64

In [ ]:
# # df_analyst_one, y_pred_df
# all_prediction = pd.merge(y_pred_df,df_analyst_three, left_on= 'tic', right_on='ticker', how='inner')
# all_prediction['medest'] = all_prediction['medest']

,tic,y_pred,E_deflat,index,ticker,predict_year,fpedats,statpers,actual,medest,gvkey,TICKER
0,CENX,-0.137355,1.673454,1272,CENX,2021.0,2021-12-31,2019-06-20,-0.62,1.27,62516,CENX
1,JKS,-0.502803,1.532952,498,JKS,2021.0,2021-12-31,2019-06-20,1.84,1.13,184182,JKS
2,D,4.874187,1.329341,398,D,2021.0,2021-12-31,2019-06-20,3.86,4.66,4029,D
3,MTD,17.093470,39.88247,2178,MTD,2021.0,2021-12-31,2019-06-20,34.01,28.50,65772,MTD
4,AZPN,3.400586,0.884936,1483,AZPN,2021.0,2021-06-30,2019-06-20,5.20,4.08,30870,AZPN
...,...,...,...,...,...,...,...,...,...,...,...,...
552,DRRX,-0.254264,-1.550713,1311,DRRX,2021.0,2021-12-31,2019-06-20,-1.60,-1.00,140044,DRRX
553,FSLR,3.783309,-3.075519,1571,FSLR,2021.0,2021-12-31,2019-06-20,4.38,4.01,175404,FSLR
554,PEP,4.619980,7.042847,2223,PEP,2021.0,2021-12-31,2019-06-20,6.26,6.44,8479,PEP
555,CDXS,-0.167413,-0.462309,144,CDXS,2021.0,2021-12-31,2019-06-20,-0.33,0.20,179841,CDXS
